In [1]:
import time
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import subprocess
from torch.nn.utils.rnn import pack_padded_sequence
import argparse
from torch.optim.lr_scheduler import StepLR
import datetime

from models_CNN_toplayan import CNN_Encoder

from models_RSICCformerDfusion_CNN_toplayan import *
from datasets import *
from eval_v2_CNN_toplayan import evaluate_transformer
import sys

from clip4cc import model_arrange

from utils import *

from torchvision.transforms.functional import to_pil_image

In [2]:
#encoder_clip_path = "C:/Users/AliCan/Desktop/clip4idc/trained_model_4090/pytorch_model.bin.7"
encoder_clip_path = "C:/Users/AliCan/Desktop/clip4idc/ckpts/caption/pytorch_model.bin.9"
encoder_clip_path = "C:/Users/AliCan/Desktop/clip4idc/ckpts/retrieval/pytorch_model.bin.14"

#trained_rsformer = "C:/Users/AliCan/Desktop/CLIP_RSICCformer/adam_optimizer_clip_4090_model_and_rsiccformer/model.tar"
trained_rsformer = "C:/Users/AliCan/Desktop/CLIP_RSICCformer/model_sonucları/RGB_CLIP(RGB)/model_dir/NEW_BEST_checkpoint_clip4IDC_MCCFormers_diff_as_Q_trans.pth.tar"
trained_rsformer = "C:/Users/AliCan/Desktop/CLIP_RSICCformer/model_sonucları/20250520_RSICCformerRGB_0/model_dir/NEW_BEST_checkpoint_clip4IDC_MCCFormers_diff_as_Q_trans.pth.tar"

encoder_image_model = model_arrange.load_model(encoder_clip_path)
check_point = torch.load(trained_rsformer, weights_only=False)

Model loaded from %s C:/Users/AliCan/Desktop/clip4idc/ckpts/retrieval/pytorch_model.bin.14


Stage-One:True, Stage-Two:False
	 embed_dim: 512
	 image_resolution: 224
	 vision_layers: 12
	 vision_width: 768
	 vision_patch_size: 32
	 context_length: 77
	 vocab_size: 49408
	 transformer_width: 512
	 transformer_heads: 8
	 transformer_layers: 12
		 linear_patch: 2d
	 cut_top_layer: 0
Initializing the model from state_dict............................
The following weights were not initialized from the pretrained model: 
   clip.visual_fusion.fusion_layer.weight
   clip.visual_fusion.fusion_layer.bias
The following weights from the pretrained model were not used: 
   clip.input_resolution
   clip.context_length
   clip.vocab_size
Model initialized from state_dict..................****..........


1
1
2
2
3
3


In [3]:
check_point.keys()

dict_keys(['epoch', 'epochs_since_improvement', 'bleu-4', 'encoder_image', 'encoder_feat', 'decoder', 'encoder_image_optimizer', 'encoder_feat_optimizer', 'decoder_optimizer', 'clip_encoder'])

In [14]:
import os
import datetime
import argparse
import sys

# Klasör oluşturma
dosya_index = 0
current_date = datetime.date.today().strftime("%Y%m%d")
folder_path = f"./model_sonucları/{current_date}_RSICCformerRGB_{dosya_index}"
while os.path.exists(folder_path):
    print(f"Folder '{folder_path}' already exists.")
    dosya_index += 1
    folder_path = f"./model_sonucları/{current_date}_RSICCformerRGB_{dosya_index}"
#os.makedirs(folder_path)
#print(f"Folder '{folder_path}' created successfully.")

# Jupyter uyumlu hale getirme
if 'ipykernel' in sys.argv[0]:
    sys.argv = ['']  # Argparse'ın kafası karışmasın diye temizle

parser = argparse.ArgumentParser(description="Image_Change_Captioning")
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# Data parameters
parser.add_argument(
    "--data_folder",
     default=r"C:\Users\AliCan\Desktop\createdFileBlackAUG",
    help="folder with data files saved by create_input_files.py.",
)
parser.add_argument(
    "--data_name", default="LEVIR_CC_5_cap_per_img_10_min_word_freq", help="base name shared by data files."
)

# Model parameters
parser.add_argument("--encoder_image_model", default="clip4IDC", help="which model does encoder use?")
parser.add_argument("--encoder_feat", default="MCCFormers_diff_as_Q")
parser.add_argument("--decoder", default="trans")
parser.add_argument("--n_heads", type=int, default=8, help="Multi-head attention in Transformer.")
parser.add_argument("--n_layers", type=int, default=3)
parser.add_argument("--decoder_n_layers", type=int, default=1)
parser.add_argument("--feature_dim_de", type=int, default=1024)
parser.add_argument("--dropout", type=float, default=0.5, help="dropout")

# Training parameters
parser.add_argument("--epochs", type=int, default=40)
parser.add_argument("--stop_criteria", type=int, default=25)
parser.add_argument("--batch_size", type=int, default=26)
parser.add_argument("--print_freq", type=int, default=100)
parser.add_argument("--workers", type=int, default=0)
parser.add_argument("--encoder_lr", type=float, default=5e-5)
parser.add_argument("--decoder_lr", type=float, default=5e-5)
parser.add_argument("--grad_clip", type=float, default=5.0)
parser.add_argument("--fine_tune_encoder", type=bool, default=True)
parser.add_argument('--encoder_image', default="resnet101", help='which model does encoder use?')


# Diğer
parser.add_argument("--checkpoint", default="None", help="path to checkpoint, None if none.")
parser.add_argument("--Split", default="VAL")
parser.add_argument("--beam_size", type=int, default=3)
parser.add_argument("--savepath", default=folder_path)

# CUDA hata takibi
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

args = parser.parse_args()


In [5]:
word_map_file = os.path.join(args.data_folder, "WORDMAP_" + args.data_name + ".json")
with open(word_map_file, "r") as j:
    word_map = json.load(j)

In [6]:
encoder_image_dim = 1024  # resnet101
encoder_image = CNN_Encoder(NetType=args.encoder_image, method=args.decoder)

encoder_feat = MCCFormers_diff_as_Q(
    feature_dim=encoder_image_dim,
    dropout=0.5,
    h=14,
    w=14,
    d_model=512,
    n_head=args.n_heads,
    n_layers=args.n_layers,
)


decoder = DecoderTransformer(
    feature_dim=args.feature_dim_de,
    vocab_size=len(word_map),
    n_head=args.n_heads,
    n_layers=args.decoder_n_layers,
    dropout=args.dropout,
)

c:\Users\AliCan\Desktop\CLIP_RSICCformer\.venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\AliCan\Desktop\CLIP_RSICCformer\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


encoder_n_layers= 3
decoder_n_layers= 1


In [7]:
def load_model_weights(model, state_or_model):
    if isinstance(state_or_model, dict):
        model.load_state_dict(state_or_model,strict=False)
    else:
        model.load_state_dict(state_or_model.state_dict())

In [8]:
load_model_weights(encoder_image, check_point["encoder_image"])
load_model_weights(encoder_feat, check_point["encoder_feat"])
load_model_weights(decoder, check_point["decoder"])
#load_model_weights(encoder_image_model,check_point['clip_encoder'])


encoder_image_model.to(device)

CLIP4IDC(
  (clip): CLIP(
    (visual): VisualTransformer(
      (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
      (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (ln_mid): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (transformer): Transformer(
        (resblocks): Sequential(
          (0): ResidualAttentionBlock(
            (attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
            )
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (mlp): Sequential(
              (c_fc): Linear(in_features=768, out_features=3072, bias=True)
              (gelu): QuickGELU()
              (c_proj): Linear(in_features=3072, out_features=768, bias=True)
            )
            (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          )
          (1): ResidualAttentionBlock(
            (attn): Mult

In [9]:
torch.cuda.is_available()

True

In [10]:
metrics, nochange_metrics, change_metrics = evaluate_transformer(
            args, encoder_image=encoder_image,clip_encoder_image=encoder_image_model, encoder_feat=encoder_feat, decoder=decoder
        )

VAL EVALUATING AT BEAM SIZE 1: 100%|██████████| 5950/5950 [00:53<00:00, 110.57it/s]


len(nochange_references):340
len(change_references):850
nochange_metric:
['Bleu_1', 'Bleu_2', 'Bleu_3', 'Bleu_4'] [0.851574803149271, 0.8277261848454928, 0.8148807902070978, 0.804038208472635]
ROUGE_L 0.8991642072445307
CIDEr 0.0
SPICE 0.3895404268933681
nochange_acc:0.8676470588235294
change_metric:
['Bleu_1', 'Bleu_2', 'Bleu_3', 'Bleu_4'] [0.6245387503302969, 0.44628108673729616, 0.31537036964900955, 0.2197118340814687]
ROUGE_L 0.41631449786713204
CIDEr 0.41798615826604324
SPICE 0.1777460030144032
change_acc:0.9952941176470588
............................................._..........
['Bleu_1', 'Bleu_2', 'Bleu_3', 'Bleu_4'] [0.6781599870192537, 0.5326572284475811, 0.42859992666422975, 0.3525994254096129]
ROUGE_L 0.554271557689246
CIDEr 0.8248509214782828
SPICE 0.23825869555125026
vocabsize: 484


In [15]:
metrics, nochange_metrics, change_metrics = evaluate_transformer(
            args, encoder_image=encoder_image,clip_encoder_image=encoder_image_model, encoder_feat=encoder_feat, decoder=decoder
        )

VAL EVALUATING AT BEAM SIZE 3: 100%|██████████| 5950/5950 [00:57<00:00, 103.98it/s]


len(nochange_references):340
len(change_references):850
nochange_metric:
['Bleu_1', 'Bleu_2', 'Bleu_3', 'Bleu_4'] [0.7851632047473085, 0.7395858998334653, 0.700663381440439, 0.6456712866640536]
ROUGE_L 0.8982118126207049
CIDEr 0.0
SPICE 0.2307213538095891
nochange_acc:0.8676470588235294
change_metric:
['Bleu_1', 'Bleu_2', 'Bleu_3', 'Bleu_4'] [0.6184422032057254, 0.4515462172668242, 0.32468971457975676, 0.23236417654239164]
ROUGE_L 0.4225919498147298
CIDEr 0.434725048744239
SPICE 0.17469943448969244
change_acc:0.9952941176470588
............................................._..........
['Bleu_1', 'Bleu_2', 'Bleu_3', 'Bleu_4'] [0.6531530348093555, 0.501365016301787, 0.3833804236367571, 0.28927883741971755]
ROUGE_L 0.5584833391878655
CIDEr 0.8435480201772576
SPICE 0.19070569715252006
vocabsize: 484


In [1]:
import torch
print(torch.__version__)
print(torch.cuda.get_arch_list())

2.6.0+cu124
['sm_50', 'sm_60', 'sm_61', 'sm_70', 'sm_75', 'sm_80', 'sm_86', 'sm_90']
